In [1]:
using ParallelTemperingMonteCarlo
using BenchmarkTools,DelimitedFiles,StaticArrays,Random

By the end of this module, the ptmc_run function will be complete. First we aim to split the dependencies from MCRun, then I aim to test variable function definitions.

In [10]:
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details


mc_cycles = 400000 #default 20% equilibration cycles on top


mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_ne13 = SphericalBC(radius=5.32*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)


Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[])

Wanting to test the use of splatting and varargs.

In [3]:
import ParallelTemperingMonteCarlo.MCRun.equilibration_cycle!

In [3]:
ptmc_run!(mc_states, move_strat, mc_params, pot, ensemble, results; save_ham = false,save=true)

Total number of moves per MC cycle: 13



equilibration done
equilibration done


MC loop done.
[0.0003275710166429321, 0.00034038231628987886, 0.0003541875217686626, 0.0003771192914525007, 0.0003871271199260189, 0.00041166221823725176, 0.00043725182080965515, 0.0004481145198137168, 0.0004892415372905008, 0.000515104682943868, 0.0005564389189273487, 0.0005866300755424616, 0.0006168592165216777, 0.0006944227240404881, 0.0007638579947522499, 0.0008900401243935854, 0.0009648086561921595, 0.0010102312231438865, 0.001162060369344718, 0.0015909789581521694, 0.0017710695798688687, 0.0022553539037819675, 0.0027215931181169776, 0.003094108733468708, 0.003327247352742674, 0.0031633947149170456, 0.003019056592225612, 0.002654620244466205, 0.0023918416792751244, 0.0021811717549868217, 0.0020685078870861556, 0.00194995364088873]
done


Two versions of equilibration_cycle! exist, one requires (mc_states,move_strat,mc_params,results,pot,ensemble), the other, only (pot) if the splatting works appropriately, this needs to be correctly reordered

In [6]:
initaliser = (mc_states,move_strat,mc_params,results,pot,ensemble)
mc_states,move_strat,ensemble,results,delta_en_hist,delta_r2,step,n_steps,a,v,r = equilibration_cycle!(initaliser...)

(MCState{Float64, 13, SphericalBC{Float64}}[MCState{Float64, 13, SphericalBC{Float64}}(5.0, 63154.997052619445, Config{13, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[5.339202509675499, 1.7547286276557148, 0.9552945639202041], [3.8235622875401982, -4.036693324695786, 1.2586922223528678], [3.8432525502537267, -1.2169635315645375, -4.030786193502644], [1.8515104475315411, 4.369051328639683, -3.1594504692142205], [1.8196441394724319, -0.19336921007065874, 5.399104677171738], [0.6008527770958447, 5.001668105430829, 2.668542641160762], [-5.339209779512827, -1.754731061649526, -0.9552951725503948], [-0.6008496122188508, -5.001669753738395, -2.668542209586626], [-1.851508499387977, -4.369053575565342, 3.1594501498809775], [-1.8196475277773039, 0.19336884278473446, -5.399104511708638], [-3.823559204867802, 4.036697652557049, -1.2586918476896083], [-3.8432544820617607, 1.2169644600549232, 4.030785366187147], [4.394257284152319e-6, 1.4401613097476533e-6, 7.841053896189476e-7]], Spherical

# Process Results

The last portion of mc_run is to analyse results. It's probably best to combine these tasks into a function

--complete

# Splatted Input

In [3]:
#(mc_states,move_strat,mc_params,results,pot,ensemble)
import ParallelTemperingMonteCarlo.MCRun.equilibration_cycle!

In [7]:
function testing_run(inputs ; save::Bool=true,save_dir = pwd())
    
    mc_states,mc_params,move_strat,ensemble,results,delta_en_hist,delta_r2,start_counter,n_steps,a,v,r = equilibration_cycle!(inputs...)
    save_states(mc_params,mc_states,0,save_dir,move_strat,ensemble)
    for i = start_counter:mc_params.mc_cycles
        @inbounds ptmc_cycle!(mc_states,results,move_strat, mc_params, pot, ensemble ,n_steps ,a ,v ,r,save,i,save_dir,delta_en_hist,delta_r2)
    end
    println("MC loop done.")
    #Evaluation

    results = finalise_results(mc_states,mc_params,results)
    return 
end

testing_run (generic function with 1 method)

In [11]:
inputs = mc_states,move_strat,mc_params,results,pot,ensemble
testing_run(inputs ;save=true)

Total number of moves per MC cycle: 13



equilibration done


MC loop done.
[0.0003275710166429321, 0.00034038231628987886, 0.0003541875217686626, 0.0003771192914525007, 0.0003871271199260189, 0.00041166221823725176, 0.00043725182080965515, 0.0004481145198137168, 0.0004892415372905008, 0.000515104682943868, 0.0005564389189273487, 0.0005866300755424616, 0.0006168592165216777, 0.0006944227240404881, 0.0007638579947522499, 0.0008900401243935854, 0.0009648086561921595, 0.0010102312231438865, 0.001162060369344718, 0.0015909789581521694, 0.0017710695798688687, 0.0022553539037819675, 0.0027215931181169776, 0.003094108733468708, 0.003327247352742674, 0.0031633947149170456, 0.003019056592225612, 0.002654620244466205, 0.0023918416792751244, 0.0021811717549868217, 0.0020685078870861556, 0.00194995364088873]


In [12]:
input = (pot,pwd())
testing_run(input;save=true)

MC loop done.


[0.0003230359045688948, 0.0003360343218109705, 0.000350018644779015, 0.00037312469807676183, 0.0003833005658842389, 0.00040799891704675984, 0.0004337466750364968, 0.0004447562352172535, 0.00048602939377589136, 0.0005120337195825401, 0.0005535063747286715, 0.0005838307634099802, 0.0006141829661945001, 0.0006918687932099309, 0.0007614303120729561, 0.0008877252404116683, 0.000962609273224772, 0.0010081375681801257, 0.0011600806504250434, 0.0015891094135521671, 0.001769307017131548, 0.0022537086666105936, 0.0027200741684222027, 0.003092701506225199, 0.00332597356439843, 0.003162239681720585, 0.003017987263836892, 0.002653650266442747, 0.0023909585261156887, 0.002180349836212071, 0.0020677499571230885, 0.0019492503477186354]
